In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.layers import Dense, Input
from keras.optimizers import Adam
from keras.models import Model
from keras.callbacks import ModelCheckpoint
import tensorflow_hub as hub
import warnings
warnings.filterwarnings(action='ignore')

import tokenization


In [2]:
def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [3]:
def build_model(bert_layer, max_len=512):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")

    _, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    clf_output = sequence_output[:, 0, :]
    out = Dense(1, activation='sigmoid')(clf_output)
    
    model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    model.compile(Adam(lr=1e-5), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [4]:
%%time
module_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1"
bert_layer = hub.KerasLayer(module_url, trainable=True)

Wall time: 8.47 s


In [5]:
train = pd.read_csv("../train.csv")
test = pd.read_csv("../test.csv")
submission = pd.read_csv("../sample_submission.csv")

In [6]:
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)

In [7]:
train_input = bert_encode(train.text.values, tokenizer, max_len=160)
test_input = bert_encode(test.text.values, tokenizer, max_len=160)
train_labels = train.target.values

In [8]:
model = build_model(bert_layer, max_len=160)

In [9]:
checkpoint = ModelCheckpoint('../BERT/model.h5', monitor='val_loss', save_best_only=True)

train_history = model.fit(
    train_input, train_labels,
    validation_split=0.2,
    epochs=3,
    callbacks=[checkpoint],
    batch_size=16
)

Epoch 1/3
381/381 [==============================] - 164s 374ms/step - loss: 0.4309 - accuracy: 0.8181 - val_loss: 0.3688 - val_accuracy: 0.8418
Epoch 2/3
381/381 [==============================] - 144s 377ms/step - loss: 0.2624 - accuracy: 0.8980 - val_loss: 0.3966 - val_accuracy: 0.8404
Epoch 3/3
381/381 [==============================] - 138s 363ms/step - loss: 0.1292 - accuracy: 0.9521 - val_loss: 0.4559 - val_accuracy: 0.8339


In [10]:
test_pred = model.predict(test_input)

102/102 [==============================] - 25s 227ms/step


In [11]:
submission['target'] = test_pred.round().astype(int)
submission.to_csv('../BERT/submission.csv', index=False)